In [15]:
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.6 MB/s eta 0:00:00


In [16]:
!pip install gensim

In [17]:
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD


In [18]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('샘플의 수 :',len(documents))


샘플의 수 : 11314


In [19]:
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ", regex=True)
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())


In [20]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
# NLTK로부터 불용어를 받아온다.
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.


In [22]:
print(tokenized_doc[1])


['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flintstone', 'chewables', 'bake', 'timmons']


In [23]:
print(tokenized_doc[:5])


0    [well, sure, story, seem, biased, disagree, st...
1    [yeah, expect, people, read, actually, accept,...
2    [although, realize, principle, strongest, poin...
3    [notwithstanding, legitimate, fuss, proposal, ...
4    [well, change, scoring, playoff, pool, unfortu...
Name: clean_doc, dtype: object


In [24]:
from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1])

[(52, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 2), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 2), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 1), (85, 2), (86, 1), (87, 1), (88, 1), (89, 1)]


In [25]:
print(dictionary[66])

faith


In [26]:
len(dictionary)

64281

In [27]:
import gensim
NUM_TOPICS = 20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=NUM_TOPICS, id2word=dictionary, passes=15)
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
  print(topic)

(0, '0.054*"drive" + 0.031*"scsi" + 0.028*"disk" + 0.020*"hard"')
(1, '0.011*"like" + 0.009*"would" + 0.009*"time" + 0.008*"know"')
(2, '0.019*"game" + 0.018*"team" + 0.014*"year" + 0.012*"play"')
(3, '0.021*"wire" + 0.019*"ground" + 0.016*"water" + 0.013*"cover"')
(4, '0.022*"health" + 0.020*"medical" + 0.014*"disease" + 0.013*"contest"')
(5, '0.012*"circuits" + 0.008*"tapes" + 0.008*"alomar" + 0.007*"idaho"')
(6, '0.015*"david" + 0.009*"navy" + 0.009*"radar" + 0.008*"mark"')
(7, '0.022*"space" + 0.009*"nasa" + 0.006*"research" + 0.006*"center"')
(8, '0.038*"file" + 0.018*"program" + 0.016*"output" + 0.016*"window"')
(9, '0.013*"science" + 0.010*"part" + 0.007*"theory" + 0.007*"evidence"')
(10, '0.020*"would" + 0.013*"people" + 0.012*"think" + 0.008*"know"')
(11, '0.012*"jesus" + 0.010*"people" + 0.008*"armenian" + 0.008*"said"')
(12, '0.012*"government" + 0.009*"israel" + 0.007*"people" + 0.007*"state"')
(13, '0.018*"encryption" + 0.016*"chip" + 0.015*"keys" + 0.014*"government"')
(1

In [28]:
print(ldamodel.print_topics())

[(0, '0.054*"drive" + 0.031*"scsi" + 0.028*"disk" + 0.020*"hard" + 0.016*"drives" + 0.014*"controller" + 0.011*"floppy" + 0.010*"tape" + 0.009*"power" + 0.008*"system"'), (1, '0.011*"like" + 0.009*"would" + 0.009*"time" + 0.008*"know" + 0.007*"good" + 0.007*"back" + 0.006*"well" + 0.006*"think" + 0.006*"could" + 0.005*"much"'), (2, '0.019*"game" + 0.018*"team" + 0.014*"year" + 0.012*"play" + 0.012*"games" + 0.011*"season" + 0.009*"players" + 0.009*"league" + 0.009*"hockey" + 0.008*"period"'), (3, '0.021*"wire" + 0.019*"ground" + 0.016*"water" + 0.013*"cover" + 0.013*"wiring" + 0.012*"neutral" + 0.009*"panel" + 0.008*"copies" + 0.008*"circuit" + 0.008*"electrical"'), (4, '0.022*"health" + 0.020*"medical" + 0.014*"disease" + 0.013*"contest" + 0.012*"patients" + 0.012*"pain" + 0.011*"filename" + 0.010*"drug" + 0.009*"among" + 0.009*"treatment"'), (5, '0.012*"circuits" + 0.008*"tapes" + 0.008*"alomar" + 0.007*"idaho" + 0.007*"camera" + 0.007*"init" + 0.006*"reserve" + 0.006*"redesign" + 0.

In [29]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [30]:
for i, topic_list in enumerate(ldamodel[corpus]):
  if i==5:
    break
  print(i, '번째 문서의 topic 비율은',topic_list)

0 번째 문서의 topic 비율은 [(8, 0.016161425), (9, 0.21922684), (10, 0.344684), (12, 0.40676665)]
1 번째 문서의 topic 비율은 [(0, 0.105630115), (3, 0.027630879), (10, 0.6855578), (11, 0.16010557)]
2 번째 문서의 topic 비율은 [(9, 0.019412369), (10, 0.5172258), (12, 0.44962153)]
3 번째 문서의 topic 비율은 [(0, 0.018559), (1, 0.1553358), (8, 0.048933864), (10, 0.44330612), (13, 0.32193923)]
4 번째 문서의 topic 비율은 [(1, 0.29155353), (2, 0.3725968), (8, 0.07184584), (10, 0.23435469)]


In [37]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%),
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                # 기존 열 이름과 맞춰서 새로운 행을 딕셔너리로 만듭니다.
                new_row = {
                    '가장 비중이 높은 토픽': int(topic_num),
                    '가장 높은 토픽의 비중': round(prop_topic, 4),
                    '각 토픽의 비중': topic_list
                }

                # 딕셔너리를 데이터프레임으로 변환
                new_df = pd.DataFrame([new_row])

                topic_table = pd.concat([topic_table, new_df], ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)


In [38]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.



In [41]:
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,12,0.4068,"[(8, 0.016160911), (9, 0.2192256), (10, 0.3446..."
1,1,10,0.6854,"[(0, 0.10557936), (3, 0.02763088), (10, 0.6854..."
2,2,10,0.5172,"[(9, 0.019411752), (10, 0.51724607), (12, 0.44..."
3,3,10,0.4433,"[(0, 0.018559026), (1, 0.1553451), (8, 0.04893..."
4,4,2,0.3726,"[(1, 0.291679), (2, 0.37259474), (8, 0.0718708..."
5,5,11,0.3194,"[(1, 0.23049706), (3, 0.07760891), (5, 0.04775..."
6,6,14,0.6135,"[(7, 0.024247846), (10, 0.25400662), (14, 0.61..."
7,7,10,0.4690,"[(1, 0.13540499), (9, 0.043408312), (10, 0.469..."
8,8,10,0.4150,"[(1, 0.27488965), (7, 0.046856705), (9, 0.0796..."
9,9,1,0.6020,"[(1, 0.60204965), (14, 0.34989583), (17, 0.036..."
